In [1]:
# import packages needed
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pyarrow.parquet as pq

In [2]:
pd.__version__

'1.4.2'

# Changes to the code since Kernal crashes if ran individually

In [13]:
def read_dataframe(filename):
    df = pq.read_table(filename).to_pandas()
    print('The number of columns in the dataset: ' + str(len(df.columns)))

    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # converts the duration to min
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    # removing outlier + wanting only 1-60 min inclusive

    return df

In [15]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

The number of columns in the dataset: 19
The number of columns in the dataset: 19


In [16]:
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [17]:
# For question 2 - looking for the standard deviation of duration
np.std(df_train['duration'])

42.5943442974141

In [18]:
# Filter it to 1 - 60 min inclusive
df_train = df_train[(df_train.duration >=1) & (df_train.duration<= 60)]
df_val = df_val[(df_val.duration >=1) & (df_val.duration<= 60)]

# Encoding

In [19]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [20]:
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

: 

In [ ]:
# we need dictionary before we use DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient ='records')
val_dicts = df_val[categorical + numerical].to_dict(orient ='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [ ]:
# To find the number of columns in the matrix (Question 4)
X_train.shape

(3009173, 516)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val= df_train[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_train, y_train_pred, squared=False)


LinearRegression()

In [ ]:
mean_squared_error(y_val, y_train_pred, squared=False)

7.649143388169879